In [ ]:
!pip install transformers

In [ ]:
!pip install accelerate -U
##https://discuss.huggingface.co/t/trainingargument-does-not-work-on-colab/43372/5## solve problem with import error

Import Dependencies

In [ ]:
# import transformers as tf
# from transformers import BertTokenizer, BertForSequenceClassification
# from transformers import TrainingArguments, Trainer
import pandas as pd
# import numpy as np
# import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Model and Tokenizer

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model = model.to('cuda')
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def process_data(data):

    conditions = [
        (data['label'] == 'NOTABU'),
        (data['label'] == 'EXP'),
        (data['label'] == 'IMP')
    ]

    choices = [0, 1, 2]
    data['label'] = np.select(conditions, choices, default=data['label'])

    processed_data = data[['tweet', 'label']]

    return processed_data

file = pd.read_csv("/content/drive/MyDrive/dataset/train_count_sw.csv")
data_train = process_data(file)

X = list(data_train["tweet"])
y = list(data_train["label"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="macro")
    precision = precision_score(y_true=labels, y_pred=pred, average="macro")
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.321400
1000,0.257100


TrainOutput(global_step=1324, training_loss=0.33224632516728425, metrics={'train_runtime': 211.9857, 'train_samples_per_second': 49.966, 'train_steps_per_second': 6.246, 'total_flos': 478997976973824.0, 'train_loss': 0.33224632516728425, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.5666764974594116,
 'eval_accuracy': 0.8164652567975831,
 'eval_precision': 0.555506269010361,
 'eval_recall': 0.5530804946712818,
 'eval_f1': 0.5501366210368999,
 'eval_runtime': 14.554,
 'eval_samples_per_second': 181.943,
 'eval_steps_per_second': 22.743,
 'epoch': 1.0}

In [ ]:
trainer.save_model('output/model_ep1')

In [ ]:
mod1 = BertForSequenceClassification.from_pretrained('output/model_ep1')
mod1.to('cuda')

In [ ]:
def predict_tweet(tweet, model):
    input = tokenizer(tweet,padding = True, truncation = True, return_tensors='pt').to('cuda')
    with torch.no_grad():
        output = model(**input)
        predicted_class = torch.argmax(output.logits, dim=1).item()
    return predicted_class

In [ ]:
data_test = pd.read_csv("/content/drive/MyDrive/dataset/test_count_sw.csv")
test = process_data(data_test)
test['predicted'] = test['tweet'].apply(lambda x: predict_tweet(x, mod1))
test.head(50)

In [ ]:
test.to_csv('/content/drive/MyDrive/prediction/AbuseEval/mod_ep1_pred.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/prediction/AbuseEval/mod_ep1_pred.csv')
y_gold = df["label"]
y_pred = df["predicted"]
report = classification_report(y_gold, y_pred, digits=2)
print(report)

              precision    recall  f1-score   support

           0       0.87      0.93      0.90       682
           1       0.57      0.62      0.59       106
           2       0.25      0.07      0.11        72

    accuracy                           0.82       860
   macro avg       0.56      0.54      0.53       860
weighted avg       0.78      0.82      0.80       860



In [ ]:
## REPORT EPOCH 5 ##

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/prediction/AbuseEval/mod_ep5_pred.csv')
y_gold = df["label"]
y_pred = df["predicted"]
report = classification_report(y_gold, y_pred, digits=2)
print(report)

              precision    recall  f1-score   support

           0       0.87      0.96      0.91       682
           1       0.66      0.54      0.59       106
           2       0.40      0.08      0.14        72

    accuracy                           0.84       860
   macro avg       0.64      0.53      0.55       860
weighted avg       0.80      0.84      0.81       860

